### Imports

In [1]:
## Libraries
import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch

from PIL import Image  # Load img
import torch
import torch.nn as nn
import statistics
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

## Python Files
import utils
import datas

### Setting up GPU Device

In [2]:
# Setting device on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

torch.backends.cudnn.deterministic = True

Using device: cpu



### Convert .mat to .png images for memory managment during training

In [3]:
# path2dataset = "nyu_depth_v2_labeled.mat"
# utils.mat2png(path2dataset)

### Load datas

In [4]:
root_folder = 'Datas'

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_loader, test_loader = datas.get_loader(root_folder, batch_size=32, num_datas=1449, train_test_ratio=0.9, transform=transform)

In [5]:
for indx, (images, depths, labels) in enumerate(train_loader):
    print(images.shape, depths.shape, labels.shape)
    plt.imshow(images[0].permute(1,2,0))
    plt.figure()
    plt.imshow(depths[0].permute(1,2,0))
    plt.figure()
    plt.imshow(labels[0].permute(1,2,0))
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>